<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#导入数据" data-toc-modified-id="导入数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入数据</a></span><ul class="toc-item"><li><span><a href="#分析" data-toc-modified-id="分析-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>分析</a></span><ul class="toc-item"><li><span><a href="#获取交易日日期" data-toc-modified-id="获取交易日日期-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>获取交易日日期</a></span></li></ul></li></ul></li></ul></div>

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
mpl.rcParams['axes.unicode_minus']=False #用来正常显示负号
import seaborn as sns
import numpy as np
import pandas as pd
from itertools import combinations,combinations_with_replacement
from collections import OrderedDict

import os,time,datetime,sys

import requests
import os
from lxml import etree
from urllib import parse
import urllib


from IPython.core.display import display, HTML

pd.set_option('display.max_rows',None)

# 导入数据

In [5]:
price_data = pd.read_csv("D:/news/i_Metrix_prices.csv")

## 分析

In [6]:
price_data.shape

(105864354, 9)

In [7]:
price_data.head()

,pSP_CUSIP,pSP_EXCHANGE,pSP_TICKER,pSP_DATE,pSP_VOLUME,pSP_OPEN,pSP_HIGH,pSP_LOW,pSP_CLOSE
0,013817101,N,AA,1996-01-02,2808000,13.1250,13.6875,13.0625,13.5938
1,013817101,N,AA,1996-01-03,5008800,13.5312,13.8438,13.5000,13.8125
2,013817101,N,AA,1996-01-04,4276000,13.7188,13.9375,13.5938,13.9375
3,013817101,N,AA,1996-01-05,2618400,13.8750,14.1250,13.8125,13.8438
4,013817101,N,AA,1996-01-08,3471200,13.7812,13.8750,12.8125,13.0938


In [11]:
price_data.pSP_EXCHANGE.unique()
len(price_data.pSP_EXCHANGE.unique())

array(['N     ', 'PK', 'OB', 'O', 'N', 'OS', 'A', 'MF', 'MF    ',
       'PK    ', 'O     ', 'OB    ', 'OS    ', 'A     ', 'JEREMY'],
      dtype=object)

15

In [8]:
pSP_CUSIP: identifer for each security
pSP_EXCHANGE：exchange on which the security is traded 证券被交易的交易所 这里有15个交易所
pSP_TICKER: identifer for each security
pSP_DATE: date which the price data corresponds
pSP_VOLUME: volume of the security
pSP_OPEN: opening price of the security
pSP_HIGH: high price of the security
pSP_LOW: low price of the security
pSP_CLOSE: closing price of the security


问题一：
数据量太大，我们是否要选取其中的一段时间，而不是全部？
选那一段时间呢？

问题二：
# pSP_CUSIP: identifer for each security
# pSP_TICKER: identifer for each security

pSP_CUSIP & pSP_TICKER 这2个变量对应不起来：(CUSIP Number 对应 公司/股票 的名称---不是全能对应上； 此二者的个数也是，不相等)
013817101 --- AA  yes
338487101 --- FLMP  yes
01881G106 --- AB  yes
# 30283W203 --- CHAQ.U no
# 36352H100 --- AGG.IV no
887080109 --- TMBR  yes

问题三：
Is 'pSP_EXCHANGE' the exact place (exchange) that each stock is traded?

问题四：
# What's the unit of 'pSP_VOLUME'?


SyntaxError: invalid syntax (<ipython-input-8-4fdfa2a2ad88>, line 1)

In [7]:
price_data.pSP_CUSIP.unique()

# CUSIP No. 30283W203


array(['013817101', '338487101', '01881G106', ..., '30283W203',
       '36352H100', '887080109'], dtype=object)

In [9]:
len(price_data.pSP_CUSIP.unique())

43407

In [8]:
price_data.pSP_TICKER.unique()

array(['AA    ', 'FLMP', 'AB    ', ..., 'CHAQ.U', 'AGG.IV', 'TMBR'],
      dtype=object)

In [10]:
len(price_data.pSP_TICKER.unique())

50988

In [20]:
aa = price_data.pSP_DATE[0]

In [ ]:
重复数据

In [33]:
part_price = price_data[price_data.pSP_DATE>='2010-01-01']

In [34]:
len(part_price)

51162751

In [1]:
part_price.duplicated()

NameError: name 'part_price' is not defined

### 获取交易日日期

In [4]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
pd.DatetimeIndex(start='2022-09-01',end='2022-10-10', freq=us_bd)

DatetimeIndex(['2022-09-01', '2022-09-02', '2022-09-06', '2022-09-07',
               '2022-09-08', '2022-09-09', '2022-09-12', '2022-09-13',
               '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-19',
               '2022-09-20', '2022-09-21', '2022-09-22', '2022-09-23',
               '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29',
               '2022-09-30', '2022-10-03', '2022-10-04', '2022-10-05',
               '2022-10-06', '2022-10-07'],
              dtype='datetime64[ns]', freq='C')

In [5]:
trade_date = pd.DatetimeIndex(start= '2021-01-04',end= '2022-04-22', freq=us_bd)
# date_list1 = [x.strftime('%Y-%m-%d') for x in date_list]
trade_date[0]
trade_date[-1]

Timestamp('2022-04-22 00:00:00', freq='C')

In [6]:
trade_date[0]

Timestamp('2021-01-04 00:00:00', freq='C')

In [7]:
len(trade_date)

328

In [25]:
price_data.pSP_DATE.min()
price_data.pSP_DATE.max()
date_list = pd.DatetimeIndex(start= price_data.pSP_DATE.min(),end= price_data.pSP_DATE.max(), freq=us_bd)
date_list1 = [x.strftime('%Y-%m-%d') for x in date_list]
date_list1[0]
date_list1[-1]

'1996-01-02'

'2020-05-29'

'1996-01-02'

'2020-05-29'

In [26]:
len(date_list1)

6126

In [27]:
len(price_data.pSP_DATE.unique())

6315

In [28]:
6315-6126
# 有 189 个不在交易日的数据

189

In [ ]:
怎么处理不在交易日的数据

In [21]:
b = pd.to_datetime(aa)
b

Timestamp('1996-01-02 00:00:00')